Need to run spark start session?: 

these notebooks share the same Iceberg storage so tables can be call from other notebooks

**Read data from csv files**

import org.apache.spark.sql.functions.{broadcast, split, lit}   
== In python: from pyspark.sql.functions import broadcast, split, lit 

In [7]:
import org.apache.spark.sql.functions.{broadcast, split, lit}


val matchesBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/matches.csv")
val matchDetailsBucketed =  spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/match_details.csv")


import org.apache.spark.sql.functions.{broadcast, split, lit}
matchesBucketed: org.apache.spark.sql.DataFrame = [match_id: string, mapid: string ... 8 more fields]
matchDetailsBucketed: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 34 more fields]


**create new table: bootcamp.matches_bucketed in icebreg**

partitioned into 16 buckets by match_id

In [9]:
spark.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")
val bucketedDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
    match_id STRING,
    is_team_game BOOLEAN,
    playlist_id STRING,
    completion_date TIMESTAMP
)
USING iceberg
PARTITIONED BY (completion_date, bucket(16, match_id));
"""
spark.sql(bucketedDDL)



bucketedDDL: String =
"
CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
    match_id STRING,
    is_team_game BOOLEAN,
    playlist_id STRING,
    completion_date TIMESTAMP
)
USING iceberg
PARTITIONED BY (completion_date, bucket(16, match_id));
"
res3: org.apache.spark.sql.DataFrame = []


**append data from matchesBucketed to bootcamp.matches_bucketed**

In [ ]:
matchesBucketed.select(
    $"match_id", $"is_team_game", $"playlist_id", $"completion_date"
    )
    .write.mode("append")
    .partitionBy("completion_date")
  .bucketBy(16, "match_id").saveAsTable("bootcamp.matches_bucketed")


**create bootcamp.match_details_bucketed table in iceberg**

In [ ]:
val bucketedDetailsDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
   match_id STRING,
   player_gamertag STRING,
   player_total_kills INTEGER,
   player_total_deaths INTEGER
)
USING iceberg
PARTITIONED BY (bucket(16, match_id));
"""
spark.sql(bucketedDetailsDDL)

**insert data to bucketedDetailsDDL**

but this one not using partition from sql    
using bucketBy() instead    
     
this one will have only 16 files as it has 16 buckets

In [ ]:
matchDetailsBucketed.select(
    $"match_id", $"player_gamertag", $"player_total_kills", $"player_total_deaths")
    .write.mode("append")
  .bucketBy(16, "match_id").saveAsTable("bootcamp.match_details_bucketed")

// disable broadcast join to see if it will use bucket join because bucket join requires two large tables

In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [ ]:
// create tempview to use spark sql
matchesBucketed.createOrReplaceTempView("matches")
matchDetailsBucketed.createOrReplaceTempView("match_details")

// explain both query plans
spark.sql("""
    SELECT * 
    FROM bootcamp.match_details_bucketed mdb 
    JOIN bootcamp.matches_bucketed md 
        ON mdb.match_id = md.match_id AND md.completion_date = DATE('2016-01-01')
""").explain()


spark.sql("""
    SELECT * 
    FROM match_details mdb 
    JOIN matches md 
        ON mdb.match_id = md.match_id
""").explain()



In [ ]:
// spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "1000000000000")

// val broadcastFromThreshold = matches.as("m").join(matchDetails.as("md"), $"m.match_id" === $"md.match_id")
//   .select($"m.completion_date", $"md.player_gamertag",  $"md.player_total_kills")
//   .take(5)

// val explicitBroadcast = matches.as("m").join(broadcast(matchDetails).as("md"), $"m.match_id" === $"md.match_id")
//   .select($"md.*", split($"completion_date", " ").getItem(0).as("ds"))

// val bucketedValues = matchDetailsBucketed.as("mdb").join(matchesBucketed.as("mb"), $"mb.match_id" === $"mdb.match_id").explain()
// // .take(5)

// val values = matchDetailsBucketed.as("m").join(matchesBucketed.as("md"), $"m.match_id" === $"md.match_id").explain()

// explicitBroadcast.write.mode("overwrite").insertInto("match_details_bucketed")

// matches.withColumn("ds", split($"completion_date", " ").getItem(0)).write.mode("overwrite").insertInto("matches_bucketed")

// spark.sql(bucketedSQL)